In [1]:
# File processing
import glob
import os

# Data processing
import random
import numpy as np
from tqdm import tqdm

# Data display 
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from matplotlib.legend_handler import HandlerLine2D

# Machine learning
import torch
import torch.nn.functional as F
from torch import linalg as LAtorch
from numpy import linalg as LAnumpy
from torch_geometric.data import DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from torch_geometric.data import Data, InMemoryDataset

# Setting device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Constants

In [2]:
NB_EPOCHS = 10
BATCH_SIZE = 10
NB_BINS = 202
EMBEDDING_SIZE = 3 # Euclidean 3D space
LEARNING_RATE = 0.001
SEED = 0
LAMBDA_KABSCH = 0.1
TRAIN_DATASET_SIZE = 800
TEST_DATASET_SIZE = 200

# Trussart analysis dataset constants
TRUSSART_HIC_PATH = '../../../../experiments/data/trussart/hic_matrices/150_TADlike_alpha_150_set0.mat'
TRUSSART_STRUCTURES_PATH = '../../../../experiments/data/trussart/structure_matrices/'

# Seeds

In [3]:
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

# Trussart analysis dataset

## Hic matrices

In [4]:
trussart_hic = np.loadtxt(TRUSSART_HIC_PATH, dtype='f', delimiter='\t')
scaler = MinMaxScaler()
trussart_hic = scaler.fit_transform(trussart_hic)

## Structure matrices

In [5]:
trussart_structures = []

file_list = os.listdir(TRUSSART_STRUCTURES_PATH)
file_list = filter(lambda f: f.endswith('.xyz'), file_list)

for file_name in file_list:
    current_trussart_structure = np.loadtxt(TRUSSART_STRUCTURES_PATH + file_name, dtype='f', delimiter='\t')
    current_trussart_structure = current_trussart_structure[:,1:]
    trussart_structures.append(current_trussart_structure)

# Create dataset

In [6]:
# grab last 4 digits of the file txt name:
def last_4digits(x):
    return(x[-8:-4])

### Hic matrices

In [7]:
train_transfer_learning_hics = []

file_list = os.listdir('../../../data/synthetic_random_trussart/train/hic_matrices/')

for file_name in sorted(filter(lambda x: x.endswith('.txt'), file_list), key = last_4digits):
    current_train_transfer_learning_hic = np.loadtxt('../../../data/synthetic_random_trussart/train/hic_matrices/'\
                                                     + file_name, dtype='f', delimiter=' ')
    train_transfer_learning_hics.append(current_train_transfer_learning_hic)

In [8]:
test_transfer_learning_hics = []

file_list = os.listdir('../../../data/synthetic_random_trussart/test/hic_matrices/')

for file_name in sorted(filter(lambda x: x.endswith('.txt'), file_list), key = last_4digits):
    current_test_transfer_learning_hic = np.loadtxt('../../../data/synthetic_random_trussart/test/hic_matrices/'\
                                                     + file_name, dtype='f', delimiter=' ')
    test_transfer_learning_hics.append(current_test_transfer_learning_hic)

### Structure matrices

In [9]:
train_transfer_learning_structures = []

file_list = os.listdir('../../../data/synthetic_random_trussart/train/structure_matrices/')

for file_name in sorted(filter(lambda x: x.endswith('.txt'), file_list), key = last_4digits):
    current_train_transfer_learning_structure = \
        np.loadtxt('../../../data/synthetic_random_trussart/train/structure_matrices/'\
                                                     + file_name, dtype='f', delimiter=' ')
    train_transfer_learning_structures.append(current_train_transfer_learning_structure)

In [10]:
test_transfer_learning_structures = []

file_list = os.listdir('../../../data/synthetic_random_trussart/test/structure_matrices/')

for file_name in sorted(filter(lambda x: x.endswith('.txt'), file_list), key = last_4digits):
    current_test_transfer_learning_structure = \
        np.loadtxt('../../../data/synthetic_random_trussart/test/structure_matrices/'\
                                                     + file_name, dtype='f', delimiter=' ')
    test_transfer_learning_structures.append(current_test_transfer_learning_structure)

### Distance matrices

In [11]:
train_transfer_learning_distances = []

file_list = os.listdir('../../../data/synthetic_random_trussart/train/distance_matrices/')

for file_name in sorted(filter(lambda x: x.endswith('.txt'), file_list), key = last_4digits):
    current_train_transfer_learning_distance = \
            np.loadtxt('../../../data/synthetic_random_trussart/train/distance_matrices/'\
                                                     + file_name, dtype='f', delimiter=' ')
    train_transfer_learning_distances.append(current_train_transfer_learning_distance)

In [12]:
test_transfer_learning_distances = []

file_list = os.listdir('../../../data/synthetic_random_trussart/test/distance_matrices/')

for file_name in sorted(filter(lambda x: x.endswith('.txt'), file_list), key = last_4digits):
    current_test_transfer_learning_distance = \
            np.loadtxt('../../../data/synthetic_random_trussart/test/distance_matrices/'\
                                                     + file_name, dtype='f', delimiter=' ')
    test_transfer_learning_distances.append(current_test_transfer_learning_distance)

### Final dataset

#### Training

In [13]:
is_training = True

In [14]:
class VanillaDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(VanillaDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []
    @property
    def processed_file_names(self):
        if is_training:
            return ['synthetic_random_trussart_bi_lstm_train_data.txt']
        else:
            return ['synthetic_random_trussart_bi_lstm_test_data.txt']

    def download(self):
        pass
        
    def process(self):
        
        data_list = []
        if is_training:
            dataset_size = TRAIN_DATASET_SIZE
        else:
            dataset_size = TEST_DATASET_SIZE
        
        for i in tqdm(range(dataset_size)):
            
            if is_training:
                transfer_learning_hic = train_transfer_learning_hics[i]
                transfer_learning_structure = train_transfer_learning_structures[i]
                transfer_learning_distance_matrix = train_transfer_learning_distances[i]
            else:
                transfer_learning_hic = test_transfer_learning_hics[i]
                transfer_learning_structure = test_transfer_learning_structures[i]
                transfer_learning_distance_matrix = test_transfer_learning_distances[i]
               
            hic_matrix = torch.FloatTensor(transfer_learning_hic)
            structure_matrix = torch.FloatTensor(transfer_learning_structure)
            distance_matrix = torch.FloatTensor(transfer_learning_distance_matrix)

            data = Data(hic_matrix=hic_matrix, structure_matrix=structure_matrix, distance_matrix=distance_matrix)
            data_list.append(data)
            
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [15]:
train_dataset = VanillaDataset('../')

  0%|          | 0/800 [00:00<?, ?it/s]

Processing...


IndexError: list index out of range

In [ ]:
train_size = len(train_dataset)
train_size

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)

#### Testing

In [ ]:
is_training = False

In [ ]:
test_dataset = VanillaDataset('../')

In [ ]:
test_size = len(test_dataset)
test_size

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Bi-LSTM Neural Network

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # Encoder
        self.lstm_encoder = torch.nn.LSTM(NB_BINS, EMBEDDING_SIZE, batch_first=True, bidirectional=True)
        self.nn_encoder = torch.nn.Linear(2*EMBEDDING_SIZE, EMBEDDING_SIZE)
        
        # Initialization
        self.xavier_initializer(self.lstm_encoder)
        
    def forward(self, x, is_training=False):
        
        x = torch.reshape(x, (BATCH_SIZE, NB_BINS, NB_BINS))
        
        # Encoder
        z, (h, c) = self.lstm_encoder(x)
        z = self.nn_encoder(z)
        
        # Structure space
        z = centralize_and_normalize_torch(z)
        
        # Distance space
        w = torch.cdist(z, z, p=2)
        
        return z, w
    
    def xavier_initializer(self, module):
        
        for name, param in module.named_parameters():
            if 'bias' in name:
                torch.nn.init.constant_(param, 0.0)
            elif 'weight' in name:
                torch.nn.init.xavier_normal_(param)

# Structure analysis functions

### Torch

In [ ]:
def centralize_torch(z):
    return z - torch.repeat_interleave(torch.reshape(torch.mean(z, axis=1), (-1,1,EMBEDDING_SIZE)), NB_BINS, dim=1)

In [ ]:
def normalize_torch(z):
    
    norms = LAtorch.norm(z, 2, dim=2)
    max_norms, _ = torch.max(norms, axis=1)
    max_norms = torch.reshape(max_norms, (BATCH_SIZE,1,1))
    max_norms = torch.repeat_interleave(max_norms, NB_BINS, dim=1)
    max_norms = torch.repeat_interleave(max_norms, EMBEDDING_SIZE, dim=2)
    max_norms[max_norms == 0] = 1
    
    return z / max_norms

In [ ]:
def centralize_and_normalize_torch(z):
    
    # Translate
    z = centralize_torch(z)
    
    # Scale
    z = normalize_torch(z)
    
    return z

### Numpy

In [ ]:
def centralize_numpy(z):
    return z - np.mean(z, axis=0)

In [ ]:
def normalize_numpy(z):
    
    norm = LAnumpy.norm(z, 2, axis=1)
    max_norm = np.max(norm, axis=0)
    if max_norm == 0:
        max_norm = 1
    
    return z / max_norm

In [ ]:
def centralize_and_normalize_numpy(z):
    
    # Translate
    z = centralize_numpy(z)
    
    # Scale
    z = normalize_numpy(z)
    
    return z

In [ ]:
def kabsch_superimposition_numpy(pred_structure, true_structure):
    
    # Centralize and normalize to unit ball
    pred_structure_unit_ball = centralize_and_normalize_numpy(pred_structure)
    true_structure_unit_ball = centralize_and_normalize_numpy(true_structure)
    
    # Rotation (solution for the constrained orthogonal Procrustes problem, subject to det(R) = 1)
    m = np.matmul(np.transpose(true_structure_unit_ball), pred_structure_unit_ball)
    u, s, vh = np.linalg.svd(m)
    
    d = np.sign(np.linalg.det(np.matmul(u, vh)))
    a = np.eye(EMBEDDING_SIZE)
    a[-1,-1] = d
    
    r = np.matmul(np.matmul(u, a), vh)
    
    pred_structure_unit_ball = np.transpose(np.matmul(r, np.transpose(pred_structure_unit_ball)))
    
    return pred_structure_unit_ball, true_structure_unit_ball

In [ ]:
def kabsch_distance_numpy(pred_structure, true_structure):
    
    pred_structure_unit_ball, true_structure_unit_ball = kabsch_superimposition_numpy(pred_structure, true_structure)
    
    # Structure comparison
    d = np.mean(np.sum(np.square(pred_structure_unit_ball - true_structure_unit_ball), axis=1))
    
    return d

# Losses

In [ ]:
def kabsch_loss_fct(pred_structure, true_structure):
    
    # NOTE: the two input structures should already be centralized and normalized
    
    m = torch.matmul(torch.transpose(true_structure, 1, 2), pred_structure)
    u, s, vh = torch.linalg.svd(m)
    
    d = torch.sign(torch.linalg.det(torch.matmul(u, vh)))
    a = torch.eye(EMBEDDING_SIZE).reshape((1, EMBEDDING_SIZE, EMBEDDING_SIZE)).repeat_interleave(BATCH_SIZE, dim=0)
    a[:,-1,-1] = d
    
    r = torch.matmul(torch.matmul(u, a), vh)
    
    pred_structure = torch.transpose(torch.matmul(r, torch.transpose(pred_structure, 1, 2)), 1, 2)
    
    return torch.mean(torch.sum(torch.square(pred_structure - true_structure), axis=2))

In [ ]:
distance_loss_fct = torch.nn.MSELoss()

In [ ]:
def compute_trussart_test_kabsch_loss():
    
    torch_trussart_hic = torch.FloatTensor(trussart_hic)
    torch_trussart_hic = torch.reshape(torch_trussart_hic, (1, NB_BINS, NB_BINS))
    torch_trussart_hic = torch.repeat_interleave(torch_trussart_hic, BATCH_SIZE, 0)
    
    trussart_pred_structure, _ = model(torch_trussart_hic)
    trussart_pred_structure = trussart_pred_structure.detach().numpy()[0]
    
    kabsch_distances = []
    for true_structure in trussart_structures:
        kabsch_distances.append(kabsch_distance_numpy(trussart_pred_structure, true_structure))
        
    return np.mean(kabsch_distances)

# Train and test

In [ ]:
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def plot_grad_flow(named_parameters):
    '''Plots the gradients flowing through different layers in the net during training.
    Can be used for checking for possible gradient vanishing / exploding problems.
    
    Usage: Plug this function in Trainer class after loss.backwards() as 
    "plot_grad_flow(self.model.named_parameters())" to visualize the gradient flow'''
    ave_grads = []
    max_grads= []
    layers = []
    for n, p in named_parameters:
        if(p.requires_grad) and ("bias" not in n):
            layers.append(n)
            ave_grads.append(p.grad.abs().mean())
            max_grads.append(p.grad.abs().max())
    plt.bar(np.arange(len(max_grads)), max_grads, alpha=0.1, lw=1, color="c")
    plt.bar(np.arange(len(max_grads)), ave_grads, alpha=0.1, lw=1, color="b")
    plt.hlines(0, 0, len(ave_grads)+1, lw=2, color="k" )
    plt.xticks(range(0,len(ave_grads), 1), layers, rotation="vertical")
    plt.xlim(left=0, right=len(ave_grads))
    plt.ylim(bottom = -0.001, top=0.02) # zoom in on the lower gradient regions
    plt.xlabel("Layers")
    plt.ylabel("average gradient")
    plt.title("Gradient flow")
    plt.grid(True)
    plt.legend([matplotlib.lines.Line2D([0], [0], color="c", lw=4),
                matplotlib.lines.Line2D([0], [0], color="b", lw=4),
                matplotlib.lines.Line2D([0], [0], color="k", lw=4)], 
               ['max-gradient', 'mean-gradient', 'zero-gradient'])

In [ ]:
def train():
    model.train()

    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        
        pred_structure, pred_distance = model(data.hic_matrix)
        
        true_hic = data.hic_matrix.to(device)
        
        true_structure = data.structure_matrix.to(device)
        true_structure = torch.reshape(true_structure, (BATCH_SIZE, NB_BINS, EMBEDDING_SIZE))
        
        pred_distance = torch.reshape(pred_distance, (BATCH_SIZE*NB_BINS, NB_BINS))
        true_distance = data.distance_matrix.to(device)
        
        # Kabsch loss
        kabsch_loss = kabsch_loss_fct(pred_structure, true_structure)
        
        # Distance loss 
        distance_loss = distance_loss_fct(pred_distance, true_distance)
        
        # Combine losses
        loss = LAMBDA_KABSCH * kabsch_loss + distance_loss
        
#         with torch.autograd.detect_anomaly():
        loss.backward()
        
        loss_all += data.num_graphs * loss.item()
        
        # Plot grad flow
#         plot_grad_flow(model.named_parameters())
        
        optimizer.step()
    return loss_all / len(train_dataset)

In [ ]:
def evaluate(loader):
    model.eval()

    true_hics = []
    
    pred_structures = []
    true_structures = []
    
    pred_distances = []
    true_distances = []
    
    kabsch_losses = []
    distance_losses = []

    with torch.no_grad():
        for data in loader:

            data = data.to(device)
            
            pred_structure, pred_distance = model(data.hic_matrix)
            
            pred_structure = pred_structure.detach().cpu()
            pred_distance = pred_distance.detach().cpu()
            
            pred_distance = torch.reshape(pred_distance, (BATCH_SIZE*NB_BINS, NB_BINS))
            
            true_hic = data.hic_matrix.detach().cpu()
            true_structure = data.structure_matrix.detach().cpu()
            true_distance = data.distance_matrix.detach().cpu()
            
            true_structure = torch.reshape(true_structure, (BATCH_SIZE, NB_BINS, EMBEDDING_SIZE))
            
            # Kabsch loss
            kabsch_loss = kabsch_loss_fct(pred_structure, true_structure).numpy()
            kabsch_losses.append(kabsch_loss)
            
            # Distance 
            distance_loss = distance_loss_fct(pred_distance, true_distance).numpy()
            distance_losses.append(distance_loss)
            
            # To numpy
            true_hic = true_hic.numpy()
            
            pred_structure = pred_structure.numpy()
            true_structure = true_structure.numpy()
            
            pred_distance = pred_distance.numpy()
            true_distance = true_distance.numpy()
            
            # Store results
            true_hics.append(true_hic)
            
            pred_structures.append(pred_structure)
            true_structures.append(true_structure)
            
            pred_distances.append(pred_distance)
            true_distances.append(true_distance)
    
    # Format results
    true_hics = np.vstack(true_hics)
    
    pred_structures = np.vstack(pred_structures)
    true_structures = np.vstack(true_structures)
    
    pred_distances = np.vstack(pred_distances)
    true_distances = np.vstack(true_distances)
    
    # Compute mean losses
    mean_kabsch_loss = np.mean(np.asarray(kabsch_losses).flatten())
    mean_distance_loss = np.mean(np.asarray(distance_losses).flatten())
    
    
    return mean_kabsch_loss, mean_distance_loss, true_hics, \
            pred_structures, true_structures, pred_distances, true_distances

In [ ]:
train_kabsch_losses_all_epochs = []
train_distance_losses_all_epochs = []

test_kabsch_losses_all_epochs = []
test_distance_losses_all_epochs = []

losses = []

trussart_test_kabsch_losses_all_epochs = []

for epoch in range(1, NB_EPOCHS+1):
    loss = train()
    losses.append(loss)
    
    ### Training
    train_mean_kabsch_loss, train_mean_distance_loss, train_true_hics, \
        train_pred_structures, train_true_structures, train_pred_distances, \
            train_true_distances = evaluate(train_loader) 
    
    # Store results
    train_kabsch_losses_all_epochs.append(train_mean_kabsch_loss)    
    train_distance_losses_all_epochs.append(train_mean_distance_loss)
    
    ### Testing
    test_mean_kabsch_loss, test_mean_distance_loss, test_true_hics, \
        test_pred_structures, test_true_structures, test_pred_distances, \
            test_true_distances = evaluate(test_loader) 
    
    ### Trussart test
    trussart_test_kabsch_loss = compute_trussart_test_kabsch_loss()
    
    # Store results
    test_kabsch_losses_all_epochs.append(test_mean_kabsch_loss)    
    test_distance_losses_all_epochs.append(test_mean_distance_loss)
    
    trussart_test_kabsch_losses_all_epochs.append(trussart_test_kabsch_loss)
    
    print('E: {:03d}, Tr K: {:.4f}, Tr D: {:.4f}, Te K: {:.4f}, Te D: {:.4f}, Trus K: {:.4f}'.format(\
        epoch, train_mean_kabsch_loss, train_mean_distance_loss, \
            test_mean_kabsch_loss, test_mean_distance_loss, trussart_test_kabsch_loss))

In [ ]:
plt.plot(losses, label='Losses')
plt.legend()

plt.show()

In [ ]:
plt.plot(train_kabsch_losses_all_epochs, label='Train Kabsch')
plt.plot(test_kabsch_losses_all_epochs, label='Test Kabsch')

plt.legend()
plt.show()

In [ ]:
plt.plot(train_distance_losses_all_epochs, label='Train Dist')
plt.plot(test_distance_losses_all_epochs, label='Test Dist')

plt.legend()
plt.show()

In [ ]:
plt.plot(trussart_test_kabsch_losses_all_epochs, label='Test Trussart Kabsch')

plt.legend()
plt.show()

# Model evaluation

In [ ]:
GRAPH_TESTED = 0

### Test distance matrix

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15,15))

ground_truth_matrix = test_true_distances[GRAPH_TESTED*NB_BINS:GRAPH_TESTED*NB_BINS+NB_BINS, :]
axes[0].imshow(ground_truth_matrix, cmap='hot', interpolation='nearest')

reconstruction_matrix = test_pred_distances[GRAPH_TESTED*NB_BINS:GRAPH_TESTED*NB_BINS+NB_BINS, :]
axes[1].imshow(reconstruction_matrix, cmap='hot', interpolation='nearest')

plt.show()

### Test latent space

In [ ]:
fig = plt.figure(figsize=(50, 50))

test_true_structure = test_true_structures[GRAPH_TESTED]
test_pred_structure = test_pred_structures[GRAPH_TESTED]

test_pred_structure_superposed, test_true_structure_superposed = \
        kabsch_superimposition_numpy(test_pred_structure, test_true_structure)

x_pred = test_pred_structure_superposed[:, 0]  
y_pred = test_pred_structure_superposed[:, 1]
z_pred = test_pred_structure_superposed[:, 2]

x_true = test_true_structure_superposed[:, 0]  
y_true = test_true_structure_superposed[:, 1]
z_true = test_true_structure_superposed[:, 2]

# Initialize figure with 4 3D subplots
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}]])

# adding surfaces to subplots.
fig.add_trace(
    go.Scatter3d(
    x=x_true, y=y_true, z=z_true,
    marker=dict(
        size=4,
        color=np.asarray(range(len(x_true))),
        colorscale='Viridis',
    ),
    line=dict(
        color='darkblue',
        width=2
    )
), row=1, col=1)

fig.add_trace(
    go.Scatter3d(
    x=x_pred, y=y_pred, z=z_pred,
    marker=dict(
        size=4,
        color=np.asarray(range(len(x_pred))),
        colorscale='Viridis',
    ),
    line=dict(
        color='darkblue',
        width=2
    )
),row=1, col=2)

fig.update_layout(
    height=1000,
    width=1000
)

fig.show()

# Shape comparison
print('Kabsch distance is ' + str(kabsch_distance_numpy(test_pred_structure, test_true_structure)))

In [ ]:
# Trussart perfect structure
trussart_true_structure = np.mean(trussart_structures, axis=0)

# Trussart predicted structure
torch_trussart_hic = torch.FloatTensor(trussart_hic)
torch_trussart_hic = torch.reshape(torch_trussart_hic, (1, NB_BINS, NB_BINS))
torch_trussart_hic = torch.repeat_interleave(torch_trussart_hic, BATCH_SIZE, 0)

trussart_pred_structure, _ = model(torch_trussart_hic)
trussart_pred_structure = trussart_pred_structure.detach().numpy()[0]

# Superpose structure using Kabsch algorithm
trussart_pred_structure_superposed, trussart_true_structure_superposed = \
        kabsch_superimposition_numpy(trussart_pred_structure, trussart_true_structure)

# Plot and compare the two structures
x_pred = trussart_pred_structure_superposed[:, 0]  
y_pred = trussart_pred_structure_superposed[:, 1]
z_pred = trussart_pred_structure_superposed[:, 2]

x_true = trussart_true_structure_superposed[:, 0]  
y_true = trussart_true_structure_superposed[:, 1]
z_true = trussart_true_structure_superposed[:, 2]

# Initialize figure with 4 3D subplots
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}]])

# adding surfaces to subplots.
fig.add_trace(
    go.Scatter3d(
    x=x_true, y=y_true, z=z_true,
    marker=dict(
        size=4,
        color=np.asarray(range(len(x_true))),
        colorscale='Viridis',
    ),
    line=dict(
        color='darkblue',
        width=2
    )
), row=1, col=1)

fig.add_trace(
    go.Scatter3d(
    x=x_pred, y=y_pred, z=z_pred,
    marker=dict(
        size=4,
        color=np.asarray(range(len(x_pred))),
        colorscale='Viridis',
    ),
    line=dict(
        color='darkblue',
        width=2
    )
),row=1, col=2)

fig.update_layout(
    height=1000,
    width=1000
)

fig.show()

In [ ]:
kabsch_distances = []

for graph_index in range(test_size):

    test_true_structure = test_true_structures[graph_index,:,:]
    test_pred_structure = test_pred_structures[graph_index,:,:]
    
    d = kabsch_distance_numpy(test_pred_structure, test_true_structure)
    kabsch_distances.append(d)

In [ ]:
n, bins, patches = plt.hist(kabsch_distances, 100, facecolor='blue', alpha=0.5)
plt.show()

print('mean: ' + str(np.mean(kabsch_distances)))
print('median: ' + str(np.median(kabsch_distances)))
print('variance: ' + str(np.var(kabsch_distances)))

# Save test losses

In [ ]:
RESULTS_ROOT = '../../../results/synthetic_random_trussart/bi_lstm/'
LAMBDA_CONFIGURATION = str(LAMBDA_KABSCH)

In [ ]:
np.savetxt(RESULTS_ROOT + 'synthetic_random_trussart_bi_lstm_losses_150_' + LAMBDA_CONFIGURATION + '.txt', losses)

np.savetxt(RESULTS_ROOT + 'synthetic_random_trussart_bi_lstm_train_kabsch_losses_all_epochs_150_' + 
           LAMBDA_CONFIGURATION + '.txt', train_kabsch_losses_all_epochs)
np.savetxt(RESULTS_ROOT + 'synthetic_random_trussart_bi_lstm_test_kabsch_losses_all_epochs_150_' + 
           LAMBDA_CONFIGURATION + '.txt', test_kabsch_losses_all_epochs)

np.savetxt(RESULTS_ROOT + 'synthetic_random_trussart_bi_lstm_train_distance_losses_all_epochs_150_' +
           LAMBDA_CONFIGURATION + '.txt', train_distance_losses_all_epochs)
np.savetxt(RESULTS_ROOT + 'synthetic_random_trussart_bi_lstm_test_distance_losses_all_epochs_150_' + 
           LAMBDA_CONFIGURATION + '.txt', test_distance_losses_all_epochs)

np.savetxt(RESULTS_ROOT + 'synthetic_random_trussart_bi_lstm_trussart_test_kabsch_losses_all_epochs_150_' +
               LAMBDA_CONFIGURATION + '.txt', trussart_test_kabsch_losses_all_epochs)
np.savetxt(RESULTS_ROOT + 'synthetic_random_trussart_bi_lstm_trussart_test_structure_150_' +
               LAMBDA_CONFIGURATION + '.txt', trussart_pred_structure)

# Save trained model

In [ ]:
torch.save(model, 
           '../../../models/synthetic_random/bi_lstm/synthetic_random_trussart_bi_lstm_model_lambda_150_' 
           + LAMBDA_CONFIGURATION + '.pt')